# AES侧信道攻击

### 导入osrtoolkit示波器模块和波形存储类

In [ ]:
from osrtoolkit.oscilloscope import Pico3000
from osrtoolkit.trace import ETSStorer

### 配置示波器控制模块

In [ ]:
pico = Pico3000()  # 声明pico示波器对象
pico.form  # 打开pico示波器配置界面

### 配置波形存储类

In [ ]:
ets = ETSStorer('AES128_mega.ets', open_type='w')  # open_type = 'w' or 'a'，如果已经存在文件， 'w' 将覆盖, ‘a’ 将在原文件进行追加 

### 与 TOE 进行通信

In [ ]:
import serial
import numpy as np
import time

In [ ]:
mega = serial.Serial('com8', 115200, timeout=1)    # 设置板子串口

In [ ]:
def get_meta():
    p = np.random.randint(0, 256, 16, dtype=np.uint8)
    send = p.tobytes()
    mega.write(send)
    recv = mega.read(16)
    c = np.frombuffer(recv, dtype=np.uint8)
    return p, c

#### 获得一次meta数据

In [ ]:
p,c = get_meta()
p,c

### 触发示波器采集曲线并查看

#### secquence采集

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams['figure.figsize'] = 20,6

In [ ]:
pico.arm()  # 将示波器置于等待触发模式，等待一次触发时间进行采集

In [ ]:
for i in range(pico.sequence_num):
    get_meta()# 控制TOE进行加密，完成对示波器的触发

In [ ]:
trs = pico.acquire_samples()  # 从pico示波器取回数据

In [ ]:
plt.plot(trs[0][0].T)
plt.grid()
plt.show()

### 波形存储

#### 以secquence模式采集曲线

In [ ]:
from tqdm import tnrange
import time

for i in tnrange(10):    # 设置secquence数
    # 采集一个secquence的数据
    pico.arm()
    plaintext = np.empty((pico.sequence_num, 16), dtype=np.uint8)
    ciphertext = np.empty((pico.sequence_num, 16), dtype=np.uint8)
    flag = True
    for j in range(pico.sequence_num):
        m = get_meta()
        plaintext[j] = m[0]
        ciphertext[j] = m[1]
    time.sleep(0.05)  # 这里必须对示波器进行一定的延迟后再进行数据拉取操作，以响应示波器时间
    trs = pico.acquire_samples()
    data = {'meta':{'plaintext': plaintext, 'ciphertext': ciphertext}, 'samples': trs}
    ets.update(data)
ets.finish()

### 对AES实现实施CPA攻击

#### 导入eshard的scared库

In [ ]:
import scared
from scared import aes

#### 定义一个选择函数，对于所有可能的密钥假设，计算第一轮字节代换(SubByte)后输出的中间值。

In [ ]:
S = aes.selection_functions.encrypt.FirstSubBytes()

#### 建立一个CPA攻击对象，参数包括选择函数，泄露模型以及判别函数。

In [ ]:
a = scared.CPAAttack(
        selection_function=S,
        model=scared.HammingWeight(),
        discriminant=scared.maxabs)

#### 攻击之前，需要一个能量迹容器将我们的TraceHeaderSet数据封装起来，同时指定攻击范围。

In [ ]:
ths = scared.traces.read_ths_from_ets_file('AES128_mega.ets')

In [ ]:
container = scared.Container(ths)

#### 执行攻击

In [ ]:
a.run(container)

In [ ]:
a.results.shape

a.results 提供计算结果，包含每一个可能的假设密钥（16个字节，每个字节有256种可能）对应的假设能量消耗值与在每一个位置记录的能量迹之间的相关系数。